In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

## Loading Dataframe

In [2]:
df = pd.read_csv("dataset.csv")
df.info()
df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33656 entries, 0 to 33655
Data columns (total 19 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   ADDRESS           33656 non-null  object 
 1   SUBURB            33656 non-null  object 
 2   PRICE             33656 non-null  int64  
 3   BEDROOMS          33656 non-null  int64  
 4   BATHROOMS         33656 non-null  int64  
 5   GARAGE            31178 non-null  float64
 6   LAND_AREA         33656 non-null  int64  
 7   FLOOR_AREA        33656 non-null  int64  
 8   BUILD_YEAR        30501 non-null  float64
 9   CBD_DIST          33656 non-null  int64  
 10  NEAREST_STN       33656 non-null  object 
 11  NEAREST_STN_DIST  33656 non-null  int64  
 12  DATE_SOLD         33656 non-null  object 
 13  POSTCODE          33656 non-null  int64  
 14  LATITUDE          33656 non-null  float64
 15  LONGITUDE         33656 non-null  float64
 16  NEAREST_SCH       33656 non-null  object

,ADDRESS,SUBURB,PRICE,BEDROOMS,BATHROOMS,GARAGE,LAND_AREA,FLOOR_AREA,BUILD_YEAR,CBD_DIST,NEAREST_STN,NEAREST_STN_DIST,DATE_SOLD,POSTCODE,LATITUDE,LONGITUDE,NEAREST_SCH,NEAREST_SCH_DIST,NEAREST_SCH_RANK
0,1 Acorn Place,South Lake,565000,4,2,2.0,600,160,2003.0,18300,Cockburn Central Station,1800,09-2018\r,6164,-32.115900,115.842450,LAKELAND SENIOR HIGH SCHOOL,0.828339,NaN
1,1 Addis Way,Wandi,365000,3,2,2.0,351,139,2013.0,26900,Kwinana Station,4900,02-2019\r,6167,-32.193470,115.859554,ATWELL COLLEGE,5.524324,129.0
2,1 Ainsley Court,Camillo,287000,3,1,1.0,719,86,1979.0,22600,Challis Station,1900,06-2015\r,6111,-32.120578,115.993579,KELMSCOTT SENIOR HIGH SCHOOL,1.649178,113.0
3,1 Albert Street,Bellevue,255000,2,1,2.0,651,59,1953.0,17900,Midland Station,3600,07-2018\r,6056,-31.900547,116.038009,SWAN VIEW SENIOR HIGH SCHOOL,1.571401,NaN
4,1 Aman Place,Lockridge,325000,4,1,2.0,466,131,1998.0,11200,Bassendean Station,2000,11-2016\r,6054,-31.885790,115.947780,KIARA COLLEGE,1.514922,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33651,9C Gold Street,South Fremantle,1040000,4,3,2.0,292,245,2013.0,16100,Fremantle Station,1500,03-2016\r,6162,-32.064580,115.751820,CHRISTIAN BROTHERS' COLLEGE,1.430350,49.0
33652,9C Pycombe Way,Westminster,410000,3,2,2.0,228,114,NaN,9600,Stirling Station,4600,02-2017\r,6061,-31.867055,115.841403,JOHN SEPTIMUS ROE ANGLICAN COMMUNITY SCHOOL,1.679644,35.0
33653,9D Pycombe Way,Westminster,427000,3,2,2.0,261,112,NaN,9600,Stirling Station,4600,02-2017\r,6061,-31.866890,115.841418,JOHN SEPTIMUS ROE ANGLICAN COMMUNITY SCHOOL,1.669159,35.0
33654,9D Shalford Way,Girrawheen,295000,3,1,2.0,457,85,1974.0,12600,Warwick Station,4400,10-2016\r,6064,-31.839680,115.842410,GIRRAWHEEN SENIOR HIGH SCHOOL,0.358494,NaN


## Feature Enginnering

In [3]:
# Renaming columns, 
df.rename(columns = {"NEAREST_SCH": "NEAREST_SCHOOL_NAME",
                     "NEAREST_SCH_DIST": "NEAREST_SCHOOL_DIST",
                     "NEAREST_SCH_RANK": "NEAREST_SCHOOL_RANK",
                     "NEAREST_STN": "NEAREST_STATION",
                     "NEAREST_STN_DIST": "NEAREST_STATION_DIST",
                     "GARAGE": "CAR_SPACES"}, inplace = True)

# Dropping columns with high cardinality,
df.drop(columns = ["ADDRESS", "NEAREST_SCHOOL_NAME"], inplace = True)

# Dropping irrelevent features,
df.drop(columns = ["LATITUDE", "LONGITUDE", "POSTCODE", "DATE_SOLD", "NEAREST_SCHOOL_RANK"], inplace = True)

# Assuming rows with empty cells for CAR_SPACES have no car spaces,
df["CAR_SPACES"] = df["CAR_SPACES"].fillna(0)

# Dropping all rows with an empty build year,
df = df.drop(df[df.BUILD_YEAR.isna() == True].index)

TARGET_FEATURE = "PRICE"
INPUT_FEATURES = {"Numerical": ["BEDROOMS", "BATHROOMS", "CAR_SPACES", "LAND_AREA", "FLOOR_AREA", "BUILD_YEAR", "CBD_DIST",
                                "NEAREST_STATION_DIST", "NEAREST_SCHOOL_DIST"],
                  "Categorical": ["SUBURB", "NEAREST_STATION"]}

# Reorganising columns,
cols = []
cols.append(TARGET_FEATURE)
for numerical_feature in INPUT_FEATURES["Numerical"]:
    cols.append(numerical_feature)
for categorical_feature in INPUT_FEATURES["Categorical"]:
    cols.append(categorical_feature)
df = df.reindex(columns = cols)

# Correcting data types,
for numerical_feature in INPUT_FEATURES["Numerical"]:
    df[numerical_feature] = df[numerical_feature].astype("float64")
for categorical_feature in INPUT_FEATURES["Categorical"]:
    df[categorical_feature] = df[categorical_feature].astype("string")
df[TARGET_FEATURE] = df[TARGET_FEATURE].astype("float64")

# Extracting features,
features = {}
for feature_name in df.drop(columns = [TARGET_FEATURE]).columns:
    features[feature_name] = []
    if df[feature_name].dtype == "string":
        for unique_value in df[feature_name].unique():
            features[feature_name].append(str(unique_value))
    else:
        pass

with open("features.txt", "w") as file:
    file.write(str(features))
file.close()

# Saving to CSV,
df.to_csv("dataset-cleaned.csv")
df.info()
df

<class 'pandas.core.frame.DataFrame'>
Index: 30501 entries, 0 to 33654
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   PRICE                 30501 non-null  float64
 1   BEDROOMS              30501 non-null  float64
 2   BATHROOMS             30501 non-null  float64
 3   CAR_SPACES            30501 non-null  float64
 4   LAND_AREA             30501 non-null  float64
 5   FLOOR_AREA            30501 non-null  float64
 6   BUILD_YEAR            30501 non-null  float64
 7   CBD_DIST              30501 non-null  float64
 8   NEAREST_STATION_DIST  30501 non-null  float64
 9   NEAREST_SCHOOL_DIST   30501 non-null  float64
 10  SUBURB                30501 non-null  string 
 11  NEAREST_STATION       30501 non-null  string 
dtypes: float64(10), string(2)
memory usage: 3.0 MB


,PRICE,BEDROOMS,BATHROOMS,CAR_SPACES,LAND_AREA,FLOOR_AREA,BUILD_YEAR,CBD_DIST,NEAREST_STATION_DIST,NEAREST_SCHOOL_DIST,SUBURB,NEAREST_STATION
0,565000.0,4.0,2.0,2.0,600.0,160.0,2003.0,18300.0,1800.0,0.828339,South Lake,Cockburn Central Station
1,365000.0,3.0,2.0,2.0,351.0,139.0,2013.0,26900.0,4900.0,5.524324,Wandi,Kwinana Station
2,287000.0,3.0,1.0,1.0,719.0,86.0,1979.0,22600.0,1900.0,1.649178,Camillo,Challis Station
3,255000.0,2.0,1.0,2.0,651.0,59.0,1953.0,17900.0,3600.0,1.571401,Bellevue,Midland Station
4,325000.0,4.0,1.0,2.0,466.0,131.0,1998.0,11200.0,2000.0,1.514922,Lockridge,Bassendean Station
...,...,...,...,...,...,...,...,...,...,...,...,...
33648,423000.0,3.0,2.0,2.0,248.0,108.0,2011.0,24900.0,5700.0,3.607062,Aubin Grove,Cockburn Central Station
33649,467000.0,4.0,2.0,2.0,400.0,132.0,2014.0,17100.0,2500.0,1.116806,Middle Swan,Midland Station
33650,955000.0,3.0,2.0,1.0,200.0,127.0,1997.0,16000.0,1900.0,1.134956,South Fremantle,Fremantle Station
33651,1040000.0,4.0,3.0,2.0,292.0,245.0,2013.0,16100.0,1500.0,1.430350,South Fremantle,Fremantle Station


## Machine Learning Dataset

In [4]:
df = pd.get_dummies(df, columns = INPUT_FEATURES["Categorical"], dtype = float)

X = df.drop(columns = [TARGET_FEATURE])
Y = df[TARGET_FEATURE]
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, random_state = 104, test_size = 0.2, shuffle = True)
df

,PRICE,BEDROOMS,BATHROOMS,CAR_SPACES,LAND_AREA,FLOOR_AREA,BUILD_YEAR,CBD_DIST,NEAREST_STATION_DIST,NEAREST_SCHOOL_DIST,...,NEAREST_STATION_Swanbourne Station,NEAREST_STATION_Thornlie Station,NEAREST_STATION_Vic,NEAREST_STATION_Warnbro Station,NEAREST_STATION_Warwick Station,NEAREST_STATION_Wellard Station,NEAREST_STATION_Welshpool Station,NEAREST_STATION_West Leederville Station,NEAREST_STATION_Whitfords Station,NEAREST_STATION_Woodbridge Station
0,565000.0,4.0,2.0,2.0,600.0,160.0,2003.0,18300.0,1800.0,0.828339,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,365000.0,3.0,2.0,2.0,351.0,139.0,2013.0,26900.0,4900.0,5.524324,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,287000.0,3.0,1.0,1.0,719.0,86.0,1979.0,22600.0,1900.0,1.649178,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,255000.0,2.0,1.0,2.0,651.0,59.0,1953.0,17900.0,3600.0,1.571401,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,325000.0,4.0,1.0,2.0,466.0,131.0,1998.0,11200.0,2000.0,1.514922,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33648,423000.0,3.0,2.0,2.0,248.0,108.0,2011.0,24900.0,5700.0,3.607062,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
33649,467000.0,4.0,2.0,2.0,400.0,132.0,2014.0,17100.0,2500.0,1.116806,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
33650,955000.0,3.0,2.0,1.0,200.0,127.0,1997.0,16000.0,1900.0,1.134956,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
33651,1040000.0,4.0,3.0,2.0,292.0,245.0,2013.0,16100.0,1500.0,1.430350,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [5]:
del df

np.savetxt("X_train.csv", X_train.to_numpy(), delimiter = ',')
np.savetxt("Y_train.csv", Y_train.to_numpy(), delimiter = ',')
np.savetxt("X_test.csv", X_test.to_numpy(), delimiter = ',')
np.savetxt("Y_test.csv", Y_test.to_numpy(), delimiter = ',')
del X_train, X_test, Y_train, Y_test